In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
from tqdm.notebook import tqdm
import glob
import datetime

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

In [12]:
# filenames = glob.glob("./FINAL/전체/*.csv")
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(filename))
# hpic = pd.concat(dfs, ignore_index=True)

# hpic["post_id"] = hpic["post_id"].astype("str")
# hpic["post_id"] = hpic["post_id"].str[-10:]

# hpic = hpic.drop(["project_id", "post_subject_size", "comment_id"], axis=1)

# hpic["post_created_at"] = hpic["post_created_at"].apply(lambda x:pd.to_datetime(x))
# hpic["comment_created_at"] = hpic["comment_created_at"].apply(lambda x:pd.to_datetime(x))
# # 가장 바깥의 따옴표를 제거합니다.
# hpic["post_to"] = hpic["post_to"].map(eval)
# hpic["post_cc"] = hpic["post_cc"].map(eval)
# hpic["to"] = hpic["post_to"] + hpic["post_cc"]

# hpic = hpic[hpic["to"].apply(len)!=0]
# hpic = hpic.drop(["post_to", "post_cc"], axis=1)

# hpic.apply(lambda x:x["to"].remove(x["post_from"]) if x["post_from"] in x["to"] else x["to"], axis=1)
# hpic = hpic[hpic["to"].apply(len)!=0]
# hpic["to"] = hpic["to"].apply(lambda x:x[0])

# hpic = hpic.rename({"post_from":"from"}, axis=1)

# hpic["from_hq"] = hpic["from"].map(email2hq)
# hpic["from_dep"] = hpic["from"].map(email2dep)
# hpic["from_name"] = hpic["from"].map(email2name)

# hpic["to_hq"] = hpic["to"].map(email2hq)
# hpic["to_dep"] = hpic["to"].map(email2dep)
# hpic["to_name"] = hpic["to"].map(email2name)

# hpic["commenter"] = hpic["commenter"].map(email2name)

# hpic["from_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["from_name"] else 0, axis=1)
# hpic["to_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["to_name"] else 0, axis=1)

# for post in set(hpic["post_id"]):
#     hpic[hpic["post_id"]==post]

# hpic["from_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["from_name"] else 0, axis=1)
# hpic["to_comment"] = hpic.apply(lambda x:x["comment_body_size"] if x["commenter"]==x["to_name"] else 0, axis=1)

# hpic = hpic[["from_hq", "from_dep", "from_name", "from_comment", "to_hq", "to_dep", "to_name", "to_comment", "comment_created_at"]]

In [13]:
# hpic.to_csv("H-PIC 멘토-멘티 댓글 작성 날짜 및 용량_-210228.csv", encoding="euc-kr")
# hpic.head()

In [3]:
# filenames = glob.glob(f"{pref}Raw Data_Collaboration Tool Usage/*.csv")
# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(filename))
# df = pd.concat(dfs, ignore_index=True)
df = pd.read_csv("D:/Github/Work/Tableau/Collaboration Tool Usage/Raw Data_Collaboration Tool Usage/협업플랫폼_전체_210701-210731.csv")

In [7]:
df = df.rename({"project_id":"prj_id", "project_name":"prj", "comment_id":"cmt_id", "comment_body_size":"cmt_body_size", "commenter":"cmter", "comment_created_at":"cmt_created_at"}, axis=1)
df[["prj_id", "post_id", "cmt_id"]] = df[["prj_id", "post_id", "cmt_id"]].astype("str")
df["post_created_at"] = pd.to_datetime(df["post_created_at"])
df["cmt_created_at"] = pd.to_datetime(df["cmt_created_at"])
# df["cmt_created_at"] = df["cmt_created_at"].apply(lambda x:pd.to_datetime(x))

df = df.sort_values(by=["post_created_at", "cmt_created_at"])

prjs = df.iloc[:, :2]
prjs = prjs.drop_duplicates(["prj_id"], keep="last")

posts = df[df["cmt_created_at"].notna()]
posts = pd.concat([posts.iloc[:, 1], posts.iloc[:, 2:9], posts.iloc[:, 12]], axis=1)
# posts = posts.drop_duplicates(["post_id", "prj"], keep="last")

cmts = pd.concat([df.iloc[:, 2], df.iloc[:, 9:]], axis=1)
cmts = cmts[cmts.notna().sum(axis=1) == 5]

prjs.to_csv("D:/Github/Work/Tableau/H-PIC/prjs.csv", index=False, encoding="euc-kr")
# posts.to_excel("D:/Github/Work/Tableau/H-PIC/posts.xlsx", index=False)
# cmts.to_excel("D:/Github/Work/Tableau/H-PIC/cmts.xlsx", index=False)

In [4]:
df = pd.read_csv("D:\Github\Work\Tableau\H-PIC\Raw Data\협업플랫폼_hpic_210701-210731.csv")

In [5]:
df

,project_id,project_name,post_id,post_subject_size,post_body_size,post_from,post_to,post_cc,post_created_at,comment_id,comment_body_size,commenter,comment_created_at
0,2765166639384511961,H-PIC(인프라수주팀),2904298405535851997,22,251,jh4793@hdc-dvp.com,['4372@hdc-dvp.com'],[],2020-12-21,3.053466e+18,827.0,jh4793@hdc-dvp.com,2021-07-15
1,2765166639384511961,H-PIC(인프라수주팀),2904298405535851997,22,251,jh4793@hdc-dvp.com,['4372@hdc-dvp.com'],[],2020-12-21,3.063684e+18,798.0,jh4793@hdc-dvp.com,2021-07-29
2,2765166639384511961,H-PIC(인프라수주팀),2904298819147886073,22,251,jh4793@hdc-dvp.com,['hadh@hdc-dvp.com'],[],2020-12-21,3.053463e+18,1605.0,jh4793@hdc-dvp.com,2021-07-15
3,2765166639384511961,H-PIC(인프라수주팀),2904298819147886073,22,251,jh4793@hdc-dvp.com,['hadh@hdc-dvp.com'],[],2020-12-21,3.063686e+18,1824.0,jh4793@hdc-dvp.com,2021-07-29
4,2765166639384511961,H-PIC(인프라수주팀),2904299133030274401,19,251,jh4793@hdc-dvp.com,['milleboy@hdc-dvp.com'],[],2020-12-21,3.053467e+18,860.0,jh4793@hdc-dvp.com,2021-07-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,3044094008847284520,H-PIC(광명4R구역-주택재개발),3044110968309559011,22,0,archigun@hdc-dvp.com,['khd0128@hdc-dvp.com'],[],2021-07-02,3.057864e+18,675.0,archigun@hdc-dvp.com,2021-07-21
1105,3044094008847284520,H-PIC(광명4R구역-주택재개발),3044111613889921411,22,0,archigun@hdc-dvp.com,['2800537@hdc-dvp.com'],[],2021-07-02,3.046430e+18,1703.0,archigun@hdc-dvp.com,2021-07-05
1106,3044094008847284520,H-PIC(광명4R구역-주택재개발),3044111613889921411,22,0,archigun@hdc-dvp.com,['2800537@hdc-dvp.com'],[],2021-07-02,3.052749e+18,971.0,2800537@hdc-dvp.com,2021-07-14
1107,3044094008847284520,H-PIC(광명4R구역-주택재개발),3044111613889921411,22,0,archigun@hdc-dvp.com,['2800537@hdc-dvp.com'],[],2021-07-02,3.052760e+18,248.0,archigun@hdc-dvp.com,2021-07-14
